In [1]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
import os

#request, pandas, pillow, tqdm


In [4]:
df =pd.read_csv("0_scrape_hm/articles.csv", dtype=str)
df.head()

,article_id,product_desc,product_url,image_url
0,1160705014,2-delat set i bomull - Ljusbeige/Vit/Blommig/V...,https://www2.hm.com/sv_se/productpage.11607050...,https://image.hm.com/assets/hm/8a/e3/8ae392b91...
1,1160705013,2-delat set i bomull - Vit/Blommig/Ljusbeige/V...,https://www2.hm.com/sv_se/productpage.11607050...,https://image.hm.com/assets/hm/b5/5b/b55ba1983...
2,1224620004,Mönstrad romper i muslin - Vit/Blommig/Vit/Blo...,https://www2.hm.com/sv_se/productpage.12246200...,https://image.hm.com/assets/hm/07/a8/07a8bb1b0...
3,1281158002,2-delat muslinset med omlott - Ljusbeige/Blomm...,https://www2.hm.com/sv_se/productpage.12811580...,https://image.hm.com/assets/hm/12/e3/12e36985d...
4,1218179012,Bodyklänning med volanger - Vit/Blommig/Crèmev...,https://www2.hm.com/sv_se/productpage.12181790...,https://image.hm.com/assets/hm/6a/98/6a9861989...


## Clean Data

### Remove items with no image

In [7]:
dir = "0_scrape_hm/images/"


valid_images = []
for index in tqdm(range(len(df))):
    subdir = f"{df['article_id'][index]}.jpg"
    if os.path.isfile(dir+subdir):
        valid_images.append(True)
    else:
        valid_images.append(False)

df["valid_image"] = valid_images
df_dropped=df[df["valid_image"]==True].copy()


#Reset index
print("items before dropping: ", len(df))
df_dropped.drop(columns="valid_image", inplace=True)
df_dropped= df_dropped.reset_index().drop(columns="index")
print("items after dropping: ", len(df_dropped))

  0%|          | 0/22741 [00:00<?, ?it/s]

items before dropping:  22741
items after dropping:  22730


## add links to images

In [8]:
"""URL = "https://d11p8vtjlacpl4.cloudfront.net/kaggle-hm-images/{}/{}.jpg"
links = [URL.format(df_dropped['article_id'][index][0:3], df_dropped['article_id'][index]) for index in range(len(df_dropped))]
df_dropped["image"] = links
df_dropped"""

#Now done on client side

'URL = "https://d11p8vtjlacpl4.cloudfront.net/kaggle-hm-images/{}/{}.jpg"\nlinks = [URL.format(df_dropped[\'article_id\'][index][0:3], df_dropped[\'article_id\'][index]) for index in range(len(df_dropped))]\ndf_dropped["image"] = links\ndf_dropped'

## Add href to h&m page

In [9]:
"""item_URL = "https://www2.hm.com/en_gb/productpage.{}.html"

item_links = []
for index in tqdm(range(len(df_dropped))):
    item_links.append(item_URL.format(df_dropped['article_id'][index]))
df_dropped["item_link"] = item_links
df_dropped"""


#Now done on client side

'item_URL = "https://www2.hm.com/en_gb/productpage.{}.html"\n\nitem_links = []\nfor index in tqdm(range(len(df_dropped))):\n    item_links.append(item_URL.format(df_dropped[\'article_id\'][index]))\ndf_dropped["item_link"] = item_links\ndf_dropped'

In [10]:
df_dropped

,article_id,prod_name,product_type_name,product_group_name,color,index_group_name
0,0108775015,Strap top,Vest top,Garment Upper body,Black,Ladieswear
1,0108775044,Strap top,Vest top,Garment Upper body,White,Ladieswear
2,0108775051,Strap top (1),Vest top,Garment Upper body,White,Ladieswear
3,0110065001,OP T-shirt (Idro),Bra,Underwear,Black,Ladieswear
4,0110065002,OP T-shirt (Idro),Bra,Underwear,White,Ladieswear
...,...,...,...,...,...,...
104830,0953450001,5pk regular Placement1,Socks,Socks & Tights,Black,Menswear
104831,0953763001,SPORT Malaga tank,Vest top,Garment Upper body,Black,Ladieswear
104832,0956217002,Cartwheel dress,Dress,Garment Full body,Black,Ladieswear
104833,0957375001,CLAIRE HAIR CLAW,Hair clip,Accessories,Black,Divided


## Save to csv

In [11]:
df_dropped.to_csv("H&M_items.csv", index=False)

In [12]:
import sys
print(f"{sys.getsizeof(df_dropped)/1000000} mb")


42.526027 mb


In [13]:
df_dropped.shape

(104835, 6)